# Preprocessing
- Stem the Data (run/running/ran --> ran)
- Identify the most commonly used words in decription
- One hot encody top top 10(ish) of the nontrvial words
- Scale the Data
- PCA (95% of variation captured)

# Models

- Linear Regression
- Decision Tree Regression
- [Plot Tree Regression](https://scikit-learn.org/stable/auto_examples/tree/plot_tree_regression.html)
- Perceptron
- Neural Network

# Set Hyperparameters

Use 10-fold cross validation to tune hyperparameters
- using RSS as an error metric?

In [3]:
import pandas as pd
import numpy as np
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords#stopwords are words like "is", and "the"
from sklearn.model_selection import train_test_split
from tqdm import tqdm #pip install tqdm, loading bar for visualization
import string#to access a convinient list of punctiation
import re#to replace numbers in strings
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer

In [4]:

df = pd.read_csv("data/winemag-data-130k-v2.csv", index_col=0)

In [5]:
df.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [29]:
for attr in df:
    print(f"{attr}: {round((sum(df[attr].isnull())/df.shape[0])*100,4)}%")

country: 0.0485%
description: 0.0%
designation: 28.8257%
points: 0.0%
price: 6.9215%
province: 0.0485%
region_1: 16.3475%
region_2: 61.1367%
taster_name: 20.1922%
taster_twitter_handle: 24.0154%
title: 0.0%
variety: 0.0008%
winery: 0.0%


In [20]:
# def text_stemming(df):
#     stop_words = set(stopwords.words("english"))
#     unhelpful_words = set(["wine", "drink"])

#     stemmer = PorterStemmer()
#     xFeat = df.to_numpy()
#     newDescription = []
#     l = len(xFeat)
#     for i in tqdm(range (l), desc="Stemming"):
#         wordList = word_tokenize(re.sub(r'\d+', 'number', (xFeat[i].lower()).translate(str.maketrans('', '', string.punctuation)))) #splits the words into a list, and lowercases them
#         description = []
#         for item in wordList:
#             if item not in stop_words:#we can get rid of stop words: "is", "a", "the"
#                 candidate = stemmer.stem(item)
#                 if candidate not in unhelpful_words:
#                     description.append(candidate) #replace each word with its stem
#         newDescription.append(description)

#     return newDescription

In [6]:
def text_stemming2(row):
    stop_words = set(stopwords.words("english"))
    unhelpful_words = set(["wine", "drink"])
    stemmer = PorterStemmer()
    wordList = word_tokenize(re.sub(r'\d+', 'number', (row.lower()).translate(str.maketrans('', '', string.punctuation))))
    wordList = [ stemmer.stem(i) for i in wordList if (i not in stop_words and i not in unhelpful_words)]

    return wordList


In [7]:
df['stemmed'] = df['description'].apply(text_stemming2)

0         [aroma, includ, tropic, fruit, broom, brimston...
1         [ripe, fruiti, smooth, still, structur, firm, ...
2         [tart, snappi, flavor, lime, flesh, rind, domi...
3         [pineappl, rind, lemon, pith, orang, blossom, ...
4         [much, like, regular, bottl, number, come, acr...
                                ...                        
129966    [note, honeysuckl, cantaloup, sweeten, delici,...
129967    [citat, given, much, decad, bottl, age, prior,...
129968    [welldrain, gravel, soil, give, crisp, dri, ch...
129969    [dri, style, pinot, gri, crisp, acid, also, we...
129970    [big, rich, offdri, power, intens, spici, roun...
Name: description, Length: 129971, dtype: object

In [18]:
df.dropna(subset=['price'], inplace=True)

In [10]:
df.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
1,Portugal,ripe,Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,tart,NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,pineappl,Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,much,Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
5,Spain,blackberri,Ars In Vitro,87,15.0,Northern Spain,Navarra,NaN,Michael Schachner,@wineschach,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem
...,...,...,...,...,...,...,...,...,...,...,...,...,...
129966,Germany,note,Brauneberger Juffer-Sonnenuhr Spätlese,90,28.0,Mosel,NaN,NaN,Anna Lee C. Iijima,NaN,Dr. H. Thanisch (Erben Müller-Burggraef) 2013 ...,Riesling,Dr. H. Thanisch (Erben Müller-Burggraef)
129967,US,citat,NaN,90,75.0,Oregon,Oregon,Oregon Other,Paul Gregutt,@paulgwine,Citation 2004 Pinot Noir (Oregon),Pinot Noir,Citation
129968,France,welldrain,Kritt,90,30.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,Gewürztraminer,Domaine Gresser
129969,France,dri,NaN,90,32.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Marcel Deiss 2012 Pinot Gris (Alsace),Pinot Gris,Domaine Marcel Deiss


# Vectorize

In [11]:
df.to_csv("data/data_stemmed.csv", index = False)

In [12]:
stemmed = pd.read_csv("data/data_stemmed.csv")
stemmed.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Portugal,ripe,Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
1,US,tart,NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
2,US,pineappl,Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
3,US,much,Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
4,Spain,blackberri,Ars In Vitro,87,15.0,Northern Spain,Navarra,NaN,Michael Schachner,@wineschach,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem


In [13]:
vectorizer = CountVectorizer(analyzer='word', tokenizer=word_tokenize, ngram_range=(1, 1), min_df=1)
print("Vectorizing...")
splitWords = vectorizer.fit_transform(stemmed['description'])
vocabMap = list(vectorizer.get_feature_names())
# Taken from http://stackoverflow.com/questions/3337301/numpy-matrix-to-array
# and http://stackoverflow.com/questions/13567345/how-to-calculate-the-sum-of-all-columns-of-a-2d-numpy-array-efficiently
counts = splitWords.sum(axis=0).A1
finalMap = Counter(dict(zip(vocabMap, counts)))
print(finalMap.most_common(10))

Vectorizing...
[('aroma', 3718), ('blend', 2942), ('made', 2413), ('ripe', 2295), ('rich', 1720), ('soft', 1610), ('light', 1546), ('number', 1413), ('there', 1252), ('fresh', 1219)]


In [14]:
print(finalMap.most_common(31))

[('aroma', 3718), ('blend', 2942), ('made', 2413), ('ripe', 2295), ('rich', 1720), ('soft', 1610), ('light', 1546), ('number', 1413), ('there', 1252), ('fresh', 1219), ('open', 1128), ('here', 1111), ('dri', 1081), ('dark', 1077), ('nose', 946), ('big', 892), ('bright', 858), ('one', 852), ('show', 813), ('earthi', 741), ('crisp', 721), ('good', 711), ('sweet', 643), ('simpl', 606), ('produc', 585), ('pretti', 583), ('intens', 579), ('beauti', 563), ('full', 542), ('smooth', 519), ('fruiti', 502)]


In [15]:
 xTrain, xTest, yTrain, yTest = train_test_split(df.loc[:, df.columns != 'price'], df['price'], test_size =.3, random_state = 334)

# Neural Network

Using RNN

In [16]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, Dataset, DataLoader, random_split
from torch.nn import functional as F

if not torch.cuda.is_available():
    raise EnvironmentError("CUDA not available, skipping neural network")

In [17]:
xTrain['description'].ravel()

array(['pink', 'pesquié', 'richli', ..., 'aroma', 'delici', 'atyp'],
      dtype=object)